# Transcript from Lecture, October 28, 2021

In [55]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2021-fall/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import json
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d




np.set_printoptions(precision = 4)

# Floating-point arithmetic is a leaky abstraction of the real numbers

In [56]:
a = 4/3
a

1.3333333333333333

In [57]:
b = a-1
b

0.33333333333333326

In [58]:
c = 3*b
c

0.9999999999999998

In [59]:
d = 1-c
d

2.220446049250313e-16

# 64-bit 2s complement integers

<b> Base 16 is "hexadecimal", with digits 0,1,2,3,...,f, four bits per digit

In [60]:
cs111.bits

{'0': '0000',
 '1': '0001',
 '2': '0010',
 '3': '0011',
 '4': '0100',
 '5': '0101',
 '6': '0110',
 '7': '0111',
 '8': '1000',
 '9': '1001',
 'a': '1010',
 'b': '1011',
 'c': '1100',
 'd': '1101',
 'e': '1110',
 'f': '1111'}

<b> int64 represents integers from $-2^{63}$ to $2^{63}-1$ using 64 bits = 16 hexadecimal digits

In [61]:
print(cs111.int64_to_hex(0))

0000000000000000


In [62]:
print(cs111.int64_to_hex(3))

0000000000000003


In [63]:
print(cs111.int64_to_hex(10))

000000000000000a


In [64]:
print(cs111.int64_to_hex(1024))

0000000000000400


In [65]:
print(cs111.int64_to_hex(-1))

ffffffffffffffff


In [66]:
print(cs111.int64_to_hex(-2))

fffffffffffffffe


In [67]:
# largest positive int64
print(cs111.int64_to_hex(2**63 - 1))

7fffffffffffffff


In [68]:
# most negative int64
print(cs111.int64_to_hex(-2**63))

8000000000000000


# IEEE Standard 64-bit floating-point

In [69]:
cs111.print_float64(0)

input     : 0
as float64: 0.0000000000000000e+00
as hex    : 0000000000000000
sign      : 0 means +
exponent  : 000 means zero or denormal



In [70]:
cs111.print_float64(1)

input     : 1
as float64: 1.0000000000000000e+00
as hex    : 3ff0000000000000
sign      : 0 means +
exponent  : 3ff means 1023 - 1023 = 0
mantissa  : 1.0000000000000000000000000000000000000000000000000000



In [71]:
cs111.print_float64(42)

input     : 42
as float64: 4.2000000000000000e+01
as hex    : 4045000000000000
sign      : 0 means +
exponent  : 404 means 1028 - 1023 = 5
mantissa  : 1.0101000000000000000000000000000000000000000000000000



In [72]:
cs111.print_float64(-1/3)

input     : -0.3333333333333333
as float64: -3.3333333333333331e-01
as hex    : bfd5555555555555
sign      : 1 means -
exponent  : 3fd means 1021 - 1023 = -2
mantissa  : 1.0101010101010101010101010101010101010101010101010101



<b> float64 has both +0 and -0, but they compare as equal

In [73]:
cs111.print_float64(-0.0)

input     : -0.0
as float64: -0.0000000000000000e+00
as hex    : 8000000000000000
sign      : 1 means -
exponent  : 000 means zero or denormal



In [74]:
-0.0 == 0.0

True

# Floating-point infinity and NaN (not-a-number)

In [75]:
np.inf

inf

In [76]:
1/np.inf

0.0

In [77]:
# This is a bad flaw in python; it should give inf!

1.0 / 0.0

ZeroDivisionError: float division by zero

In [78]:
np.inf + 100

inf

In [79]:
-2 * np.inf

-inf

In [80]:
0 * np.inf

nan

In [81]:
cs111.print_float64(np.inf)

input     : inf
as float64: inf
as hex    : 7ff0000000000000
sign      : 0 means +
exponent  : 7ff means inf or nan



In [82]:
cs111.print_float64(-np.inf)

input     : -inf
as float64: -inf
as hex    : fff0000000000000
sign      : 1 means -
exponent  : 7ff means inf or nan



In [83]:
cs111.print_float64(np.nan)

input     : nan
as float64: nan
as hex    : 7ff8000000000000
sign      : 0 means +
exponent  : 7ff means inf or nan



<b> Is infinity equal to infinity?

In [84]:
np.inf == np.inf

True

<b> NaN is not equal to anything, including itself!

In [85]:
np.nan == 0

False

In [86]:
np.nan == np.nan

False

In [87]:
np.isnan(np.nan)

True

In [88]:
np.isnan(np.inf)

False

In [89]:
np.isnan(3.14)

False

# Properties of floating-point arithmetic

<b> Numbers that get big

In [90]:
x = 1.0
while x < 2*x:
    print('x:', x, '    2x:', 2*x)
    lastx = x
    x = 2*x
print('x:', x, '    2x:', 2*x)

x: 1.0     2x: 2.0
x: 2.0     2x: 4.0
x: 4.0     2x: 8.0
x: 8.0     2x: 16.0
x: 16.0     2x: 32.0
x: 32.0     2x: 64.0
x: 64.0     2x: 128.0
x: 128.0     2x: 256.0
x: 256.0     2x: 512.0
x: 512.0     2x: 1024.0
x: 1024.0     2x: 2048.0
x: 2048.0     2x: 4096.0
x: 4096.0     2x: 8192.0
x: 8192.0     2x: 16384.0
x: 16384.0     2x: 32768.0
x: 32768.0     2x: 65536.0
x: 65536.0     2x: 131072.0
x: 131072.0     2x: 262144.0
x: 262144.0     2x: 524288.0
x: 524288.0     2x: 1048576.0
x: 1048576.0     2x: 2097152.0
x: 2097152.0     2x: 4194304.0
x: 4194304.0     2x: 8388608.0
x: 8388608.0     2x: 16777216.0
x: 16777216.0     2x: 33554432.0
x: 33554432.0     2x: 67108864.0
x: 67108864.0     2x: 134217728.0
x: 134217728.0     2x: 268435456.0
x: 268435456.0     2x: 536870912.0
x: 536870912.0     2x: 1073741824.0
x: 1073741824.0     2x: 2147483648.0
x: 2147483648.0     2x: 4294967296.0
x: 4294967296.0     2x: 8589934592.0
x: 8589934592.0     2x: 17179869184.0
x: 17179869184.0     2x: 34359738368.0

x: 3.1082702275611665e+85     2x: 6.216540455122333e+85
x: 6.216540455122333e+85     2x: 1.2433080910244666e+86
x: 1.2433080910244666e+86     2x: 2.4866161820489332e+86
x: 2.4866161820489332e+86     2x: 4.9732323640978664e+86
x: 4.9732323640978664e+86     2x: 9.946464728195733e+86
x: 9.946464728195733e+86     2x: 1.9892929456391466e+87
x: 1.9892929456391466e+87     2x: 3.978585891278293e+87
x: 3.978585891278293e+87     2x: 7.957171782556586e+87
x: 7.957171782556586e+87     2x: 1.5914343565113173e+88
x: 1.5914343565113173e+88     2x: 3.1828687130226345e+88
x: 3.1828687130226345e+88     2x: 6.365737426045269e+88
x: 6.365737426045269e+88     2x: 1.2731474852090538e+89
x: 1.2731474852090538e+89     2x: 2.5462949704181076e+89
x: 2.5462949704181076e+89     2x: 5.092589940836215e+89
x: 5.092589940836215e+89     2x: 1.018517988167243e+90
x: 1.018517988167243e+90     2x: 2.037035976334486e+90
x: 2.037035976334486e+90     2x: 4.074071952668972e+90
x: 4.074071952668972e+90     2x: 8.1481439053379

x: 9.893216058924181e+173     2x: 1.9786432117848363e+174
x: 1.9786432117848363e+174     2x: 3.9572864235696725e+174
x: 3.9572864235696725e+174     2x: 7.914572847139345e+174
x: 7.914572847139345e+174     2x: 1.582914569427869e+175
x: 1.582914569427869e+175     2x: 3.165829138855738e+175
x: 3.165829138855738e+175     2x: 6.331658277711476e+175
x: 6.331658277711476e+175     2x: 1.2663316555422952e+176
x: 1.2663316555422952e+176     2x: 2.5326633110845904e+176
x: 2.5326633110845904e+176     2x: 5.065326622169181e+176
x: 5.065326622169181e+176     2x: 1.0130653244338362e+177
x: 1.0130653244338362e+177     2x: 2.0261306488676723e+177
x: 2.0261306488676723e+177     2x: 4.052261297735345e+177
x: 4.052261297735345e+177     2x: 8.10452259547069e+177
x: 8.10452259547069e+177     2x: 1.620904519094138e+178
x: 1.620904519094138e+178     2x: 3.241809038188276e+178
x: 3.241809038188276e+178     2x: 6.483618076376552e+178
x: 6.483618076376552e+178     2x: 1.2967236152753103e+179
x: 1.296723615275310

x: 2.891790293717215e+222     2x: 5.78358058743443e+222
x: 5.78358058743443e+222     2x: 1.156716117486886e+223
x: 1.156716117486886e+223     2x: 2.313432234973772e+223
x: 2.313432234973772e+223     2x: 4.626864469947544e+223
x: 4.626864469947544e+223     2x: 9.253728939895087e+223
x: 9.253728939895087e+223     2x: 1.8507457879790174e+224
x: 1.8507457879790174e+224     2x: 3.701491575958035e+224
x: 3.701491575958035e+224     2x: 7.40298315191607e+224
x: 7.40298315191607e+224     2x: 1.480596630383214e+225
x: 1.480596630383214e+225     2x: 2.961193260766428e+225
x: 2.961193260766428e+225     2x: 5.922386521532856e+225
x: 5.922386521532856e+225     2x: 1.1844773043065711e+226
x: 1.1844773043065711e+226     2x: 2.3689546086131423e+226
x: 2.3689546086131423e+226     2x: 4.737909217226285e+226
x: 4.737909217226285e+226     2x: 9.47581843445257e+226
x: 9.47581843445257e+226     2x: 1.895163686890514e+227
x: 1.895163686890514e+227     2x: 3.790327373781028e+227
x: 3.790327373781028e+227     2

x: 2.163894399531685e+273     2x: 4.32778879906337e+273
x: 4.32778879906337e+273     2x: 8.65557759812674e+273
x: 8.65557759812674e+273     2x: 1.731115519625348e+274
x: 1.731115519625348e+274     2x: 3.462231039250696e+274
x: 3.462231039250696e+274     2x: 6.924462078501392e+274
x: 6.924462078501392e+274     2x: 1.3848924157002783e+275
x: 1.3848924157002783e+275     2x: 2.7697848314005566e+275
x: 2.7697848314005566e+275     2x: 5.539569662801113e+275
x: 5.539569662801113e+275     2x: 1.1079139325602226e+276
x: 1.1079139325602226e+276     2x: 2.2158278651204453e+276
x: 2.2158278651204453e+276     2x: 4.431655730240891e+276
x: 4.431655730240891e+276     2x: 8.863311460481781e+276
x: 8.863311460481781e+276     2x: 1.7726622920963562e+277
x: 1.7726622920963562e+277     2x: 3.5453245841927125e+277
x: 3.5453245841927125e+277     2x: 7.090649168385425e+277
x: 7.090649168385425e+277     2x: 1.418129833677085e+278
x: 1.418129833677085e+278     2x: 2.83625966735417e+278
x: 2.83625966735417e+278

In [91]:
cs111.print_float64(lastx)

input     : 8.98846567431158e+307
as float64: 8.9884656743115795e+307
as hex    : 7fe0000000000000
sign      : 0 means +
exponent  : 7fe means 2046 - 1023 = 1023
mantissa  : 1.0000000000000000000000000000000000000000000000000000



<b> Numbers that get little

In [92]:
x = 1.0
while x > x/2:
    print('x:', x, '    x/2:', 2*x)
    lastx = x
    x = x/2
print('x:', x, '    x/2:', x/2)

x: 1.0     x/2: 2.0
x: 0.5     x/2: 1.0
x: 0.25     x/2: 0.5
x: 0.125     x/2: 0.25
x: 0.0625     x/2: 0.125
x: 0.03125     x/2: 0.0625
x: 0.015625     x/2: 0.03125
x: 0.0078125     x/2: 0.015625
x: 0.00390625     x/2: 0.0078125
x: 0.001953125     x/2: 0.00390625
x: 0.0009765625     x/2: 0.001953125
x: 0.00048828125     x/2: 0.0009765625
x: 0.000244140625     x/2: 0.00048828125
x: 0.0001220703125     x/2: 0.000244140625
x: 6.103515625e-05     x/2: 0.0001220703125
x: 3.0517578125e-05     x/2: 6.103515625e-05
x: 1.52587890625e-05     x/2: 3.0517578125e-05
x: 7.62939453125e-06     x/2: 1.52587890625e-05
x: 3.814697265625e-06     x/2: 7.62939453125e-06
x: 1.9073486328125e-06     x/2: 3.814697265625e-06
x: 9.5367431640625e-07     x/2: 1.9073486328125e-06
x: 4.76837158203125e-07     x/2: 9.5367431640625e-07
x: 2.384185791015625e-07     x/2: 4.76837158203125e-07
x: 1.1920928955078125e-07     x/2: 2.384185791015625e-07
x: 5.960464477539063e-08     x/2: 1.1920928955078125e-07
x: 2.9802322387695

x: 2.7635739376302223e-76     x/2: 5.527147875260445e-76
x: 1.3817869688151111e-76     x/2: 2.7635739376302223e-76
x: 6.908934844075556e-77     x/2: 1.3817869688151111e-76
x: 3.454467422037778e-77     x/2: 6.908934844075556e-77
x: 1.727233711018889e-77     x/2: 3.454467422037778e-77
x: 8.636168555094445e-78     x/2: 1.727233711018889e-77
x: 4.3180842775472223e-78     x/2: 8.636168555094445e-78
x: 2.1590421387736112e-78     x/2: 4.3180842775472223e-78
x: 1.0795210693868056e-78     x/2: 2.1590421387736112e-78
x: 5.397605346934028e-79     x/2: 1.0795210693868056e-78
x: 2.698802673467014e-79     x/2: 5.397605346934028e-79
x: 1.349401336733507e-79     x/2: 2.698802673467014e-79
x: 6.747006683667535e-80     x/2: 1.349401336733507e-79
x: 3.3735033418337674e-80     x/2: 6.747006683667535e-80
x: 1.6867516709168837e-80     x/2: 3.3735033418337674e-80
x: 8.433758354584419e-81     x/2: 1.6867516709168837e-80
x: 4.2168791772922093e-81     x/2: 8.433758354584419e-81
x: 2.1084395886461046e-81     x/2

x: 3.2802129431479926e-142     x/2: 6.560425886295985e-142
x: 1.6401064715739963e-142     x/2: 3.2802129431479926e-142
x: 8.200532357869981e-143     x/2: 1.6401064715739963e-142
x: 4.100266178934991e-143     x/2: 8.200532357869981e-143
x: 2.0501330894674953e-143     x/2: 4.100266178934991e-143
x: 1.0250665447337477e-143     x/2: 2.0501330894674953e-143
x: 5.1253327236687384e-144     x/2: 1.0250665447337477e-143
x: 2.5626663618343692e-144     x/2: 5.1253327236687384e-144
x: 1.2813331809171846e-144     x/2: 2.5626663618343692e-144
x: 6.406665904585923e-145     x/2: 1.2813331809171846e-144
x: 3.2033329522929615e-145     x/2: 6.406665904585923e-145
x: 1.6016664761464807e-145     x/2: 3.2033329522929615e-145
x: 8.008332380732404e-146     x/2: 1.6016664761464807e-145
x: 4.004166190366202e-146     x/2: 8.008332380732404e-146
x: 2.002083095183101e-146     x/2: 4.004166190366202e-146
x: 1.0010415475915505e-146     x/2: 2.002083095183101e-146
x: 5.0052077379577523e-147     x/2: 1.001041547591550

x: 3.5910604374153675e-189     x/2: 7.182120874830735e-189
x: 1.7955302187076838e-189     x/2: 3.5910604374153675e-189
x: 8.977651093538419e-190     x/2: 1.7955302187076838e-189
x: 4.4888255467692094e-190     x/2: 8.977651093538419e-190
x: 2.2444127733846047e-190     x/2: 4.4888255467692094e-190
x: 1.1222063866923024e-190     x/2: 2.2444127733846047e-190
x: 5.611031933461512e-191     x/2: 1.1222063866923024e-190
x: 2.805515966730756e-191     x/2: 5.611031933461512e-191
x: 1.402757983365378e-191     x/2: 2.805515966730756e-191
x: 7.01378991682689e-192     x/2: 1.402757983365378e-191
x: 3.506894958413445e-192     x/2: 7.01378991682689e-192
x: 1.7534474792067224e-192     x/2: 3.506894958413445e-192
x: 8.767237396033612e-193     x/2: 1.7534474792067224e-192
x: 4.383618698016806e-193     x/2: 8.767237396033612e-193
x: 2.191809349008403e-193     x/2: 4.383618698016806e-193
x: 1.0959046745042015e-193     x/2: 2.191809349008403e-193
x: 5.479523372521008e-194     x/2: 1.0959046745042015e-193
x:

x: 1.9848322066592191e-264     x/2: 3.9696644133184383e-264
x: 9.924161033296096e-265     x/2: 1.9848322066592191e-264
x: 4.962080516648048e-265     x/2: 9.924161033296096e-265
x: 2.481040258324024e-265     x/2: 4.962080516648048e-265
x: 1.240520129162012e-265     x/2: 2.481040258324024e-265
x: 6.20260064581006e-266     x/2: 1.240520129162012e-265
x: 3.10130032290503e-266     x/2: 6.20260064581006e-266
x: 1.550650161452515e-266     x/2: 3.10130032290503e-266
x: 7.753250807262575e-267     x/2: 1.550650161452515e-266
x: 3.8766254036312874e-267     x/2: 7.753250807262575e-267
x: 1.9383127018156437e-267     x/2: 3.8766254036312874e-267
x: 9.691563509078218e-268     x/2: 1.9383127018156437e-267
x: 4.845781754539109e-268     x/2: 9.691563509078218e-268
x: 2.4228908772695546e-268     x/2: 4.845781754539109e-268
x: 1.2114454386347773e-268     x/2: 2.4228908772695546e-268
x: 6.057227193173887e-269     x/2: 1.2114454386347773e-268
x: 3.0286135965869433e-269     x/2: 6.057227193173887e-269
x: 1.5

In [93]:
cs111.print_float64(lastx)

input     : 5e-324
as float64: 4.9406564584124654e-324
as hex    : 0000000000000001
sign      : 0 means +
exponent  : 000 means zero or denormal



<b>Machine epsilon

In [94]:
x = 1.0
while 1 + x > 1:
    print('x:', x, '    1 + x:', 1+x)
    x = x/2
print('x:', x, '    1 + x:', 1+x)

x: 1.0     1 + x: 2.0
x: 0.5     1 + x: 1.5
x: 0.25     1 + x: 1.25
x: 0.125     1 + x: 1.125
x: 0.0625     1 + x: 1.0625
x: 0.03125     1 + x: 1.03125
x: 0.015625     1 + x: 1.015625
x: 0.0078125     1 + x: 1.0078125
x: 0.00390625     1 + x: 1.00390625
x: 0.001953125     1 + x: 1.001953125
x: 0.0009765625     1 + x: 1.0009765625
x: 0.00048828125     1 + x: 1.00048828125
x: 0.000244140625     1 + x: 1.000244140625
x: 0.0001220703125     1 + x: 1.0001220703125
x: 6.103515625e-05     1 + x: 1.00006103515625
x: 3.0517578125e-05     1 + x: 1.000030517578125
x: 1.52587890625e-05     1 + x: 1.0000152587890625
x: 7.62939453125e-06     1 + x: 1.0000076293945312
x: 3.814697265625e-06     1 + x: 1.0000038146972656
x: 1.9073486328125e-06     1 + x: 1.0000019073486328
x: 9.5367431640625e-07     1 + x: 1.0000009536743164
x: 4.76837158203125e-07     1 + x: 1.0000004768371582
x: 2.384185791015625e-07     1 + x: 1.000000238418579
x: 1.1920928955078125e-07     1 + x: 1.0000001192092896
x: 5.96046447753

<b> *machine epsilon* is the smallest x such that 1 + x > 1

In [98]:
eps = 2**(-52)

print('eps:')
cs111.print_float64(eps)

print('1 + eps:')
cs111.print_float64(1 + eps)

1 + eps == 1

eps:
input     : 2.220446049250313e-16
as float64: 2.2204460492503131e-16
as hex    : 3cb0000000000000
sign      : 0 means +
exponent  : 3cb means 971 - 1023 = -52
mantissa  : 1.0000000000000000000000000000000000000000000000000000

1 + eps:
input     : 1.0000000000000002
as float64: 1.0000000000000002e+00
as hex    : 3ff0000000000001
sign      : 0 means +
exponent  : 3ff means 1023 - 1023 = 0
mantissa  : 1.0000000000000000000000000000000000000000000000000001



False

In [99]:
half_eps = 2**(-53)

print('half_eps:')
cs111.print_float64(half_eps)

print('1 + half_eps:')
cs111.print_float64(1 + half_eps)

1 + half_eps == 1

half_eps:
input     : 1.1102230246251565e-16
as float64: 1.1102230246251565e-16
as hex    : 3ca0000000000000
sign      : 0 means +
exponent  : 3ca means 970 - 1023 = -53
mantissa  : 1.0000000000000000000000000000000000000000000000000000

1 + half_eps:
input     : 1.0
as float64: 1.0000000000000000e+00
as hex    : 3ff0000000000000
sign      : 0 means +
exponent  : 3ff means 1023 - 1023 = 0
mantissa  : 1.0000000000000000000000000000000000000000000000000000



True